In [ ]:
from pathlib import Path
import sys

ROOT = next((p for p in [Path.cwd(), *Path.cwd().parents] if (p / "scripts").is_dir() or (p / "data").is_dir()), None)
if ROOT is None:
    raise RuntimeError("Repo-Root not found (expected folder 'scripts' or 'data').")
sys.path.insert(0, str(ROOT))

DATA_DIR = ROOT / "data"
DRF_DIRS_BIG = [(DATA_DIR / "drf_big" / f"precomputed_drf_{m}", m) for m in ("edge", "vertex", "sp")]
DRF_DIRS_SMALL = [(DATA_DIR / "drf_small" / f"precomputed_drf_{m}", m) for m in ("edge", "vertex", "sp")]
ITS_DIRS_BIG = [(DATA_DIR / "its_big" / f"precomputed_its_{m}", m) for m in ("edge", "vertex", "sp")]
ITS_DIRS_SMALL = [(DATA_DIR / "its_small" / f"precomputed_its_{m}", m) for m in ("edge", "vertex", "sp")]

# WP3 — Kernel-based Classification (SVM)

This notebook implements kernel inner products on precomputed hashed feature sets and runs
SVM classification for DRF–WL and ITS–WL across different feature types (vertex/edge/shortest-path),
dataset sizes, numbers of classes, and train/test splits.

In [ ]:
import numpy as np
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "vscode"
import pickle
from pathlib import Path

#local imports
from scripts.wp3.wp3_loader import load_precomputed_features

from scripts.wp3.wp3_kernel import (
    build_kernel_matrix_from_loaded, 
    kernel_matrix_stats,
    kernel_multiset_intersection,
    build_kernel_matrix_from_loaded, 
    kernel_matrix_stats,
)

from scripts.wp3.wp3_svm import (
    train_svm_with_precomputed_kernel,
    run_svm_for_modes,
    train_svm_from_precomputed_dir,
)

## 1) Paths to precomputed feature directories

We load precomputed feature representations (stored as `.pkl`) for:
- DRF–WL: reactant/product difference features
- ITS–WL: features from the ITS reaction graph

Each representation is available for three feature modes: vertex, edge, shortest-path.

### Load DRF–WL Features
Load precomputed DRF–WL feature sets and reaction class labels for kernel-based classification.

In [ ]:

X_drf, y_drf = {}, {}
for path, mode in DRF_DIRS_BIG:  # ACHTUNG: Reihenfolge (path, mode)
    assert path.exists(), f"Pfad nicht gefunden: {path}"
    X, y = load_precomputed_features(path, feature_key="drf_wl")
    X_drf[mode] = X
    y_drf[mode] = y
    print(f"\nLoaded DRF features ({mode}) from {path}")
    print("Number of reactions:", len(X))
    print("Number of classes:", len(set(y)))

### Load ITS–WL Features
Load precomputed ITS–WL feature sets and reaction class labels derived from the ITS graph.

In [ ]:
X_its = {}
y_its = {}
for path, mode in ITS_DIRS_BIG:  # ACHTUNG: Reihenfolge (path, mode)
    assert path.exists(), f"Pfad nicht gefunden: {path}"
    X, y = load_precomputed_features(path, feature_key="its_wl")
    X_its[mode] = X
    y_its[mode] = y
    print(f"\nLoaded ITS features ({mode}) from {path}")
    print("Number of reactions:", len(X))
    print("Number of classes:", len(set(y)))


The output confirms that all precomputed DRF–WL feature representations
(edge, vertex, and shortest-path) were loaded successfully. Each representation
contains the full dataset of 50,000 reactions across 50 reaction classes,
providing a consistent basis for kernel computation and classification.

## 2) Kernel inner product on hash sets

The lab definition reduces all kernels to counting common elements of two hashed feature sets.
Given two reactions with feature hash sets \(S_G, S_H\), the kernel is:
\[
k(G,H) = |S_G \cap S_H|
\]

Our precomputed features are stored as Counters. For the required hashset kernel, we use the Counter keys.

Ein Kernel ist eine Funktion, die sagt, wie ähnlich zwei Reaktionen sind.

### Kernel sanity check (DRF–WL)

We verify that the multiset kernel produces meaningful similarities on the precomputed DRF–WL feature multisets.  
Self-similarity \(k(x,x)\) is clearly positive, and different reactions can still share a non-zero overlap, indicating common reaction-change patterns captured by DRF–WL.

In [ ]:
mode = "edge"   # "edge" | "vertex" | "sp"
X = X_its[mode]  # oder X_drf[mode]

# finde erstes Paar mit k>0
for i in range(len(X)):
    if len(X[i]) == 0:
        continue
    for j in range(i + 1, len(X)):
        if len(X[j]) == 0:
            continue
        k = kernel_multiset_intersection(X[i], X[j])
        if k > 0:
            print("Found non-zero kernel at:", i, j, "value:", k)
            break
    else:
        continue
    break


In [ ]:
# Finde ein nicht-leeres Paar
for i in range(len(X)):
    if len(X[i]) == 0:
        continue
    for j in range(i+1, len(X)):
        if len(X[j]) == 0:
            continue
        k = kernel_multiset_intersection(X[i], X[j])
        if k > 0:
            print("Found non-zero kernel at:", i, j, "value:", k)
            break
    else:
        continue
    break

### Kernel Matrix Construction

To apply kernel-based classification, the pairwise similarities between all reactions are computed and stored in a kernel matrix. Each entry \(K_{ij}\) represents the multiset kernel value between reactions \(i\) and \(j\). This matrix serves as the direct input for training a Support Vector Machine with a precomputed kernel.

### DRF–WL Kernel Matrix (edge features)

This heatmap visualizes the kernel matrix computed using the DRF–WL edge kernel for a subset of reactions.
Each entry \(K_{ij}\) represents the multiset intersection between the DRF–WL feature representations of reaction \(i\) and reaction \(j\).

The bright diagonal indicates high self-similarity, as each reaction shares all its features with itself.
Most off-diagonal entries are close to zero, which reflects the sparsity of the DRF representation:  
DRF removes all static molecular structure and retains only features corresponding to reaction-specific changes.

Non-zero off-diagonal values highlight pairs of reactions that share similar bond-change patterns.
This confirms that the DRF–WL kernel captures meaningful similarities between reactions while remaining highly selective.

In [ ]:
mode = "edge"   # "edge" | "vertex" | "sp"
n = 200

K_drf, y_small = build_kernel_matrix_from_loaded(
    X_drf, y_drf,
    mode=mode,
    n=n,
)

stats = kernel_matrix_stats(K_drf)
print("Kernel matrix stats:", stats)

fig = px.imshow(
    K_drf,
    title=f"Kernel Matrix Heatmap (DRF–WL {mode}, n={n})",
    aspect="auto",
)
fig.show()

**Figure (DRF–WL):** Kernel matrix heatmap computed using the DRF–WL edge kernel.
Each entry \(K_{ij}\) represents the multiset intersection between the DRF–WL feature representations of reactions \(i\) and \(j\).
The diagonal indicates self-similarity, while off-diagonal values are mostly close to zero.
This sparsity reflects the DRF representation, which removes static molecular structure and retains only reaction-specific changes.
Non-zero off-diagonal entries therefore highlight reactions with similar bond-change patterns.

#### Error Handling

In [ ]:

pkl = next(Path("drf/precomputed_drf_edge").glob("*.pkl"))
obj = pickle.load(open(pkl, "rb"))

print("Keys:", obj.keys())
print("n_errors:", obj["meta"]["n_errors"])
print("First error:", obj["errors"][:1])
print("First feature:", type(obj["drf_wl"][0]), obj["drf_wl"][0])


In [ ]:

DIR = Path("drf/precomputed_drf_edge")  # <- GENAU der Ordner, den du lädst
pkl = sorted(DIR.glob("*.pkl"))[0]
print("Inspecting:", pkl)

with open(pkl, "rb") as f:
    obj = pickle.load(f)

print("Keys:", obj.keys())
print("Meta n_rows:", obj["meta"]["n_rows"])
print("Meta n_errors:", obj["meta"]["n_errors"])
print("First error (if any):", obj["errors"][:1])

# Jetzt das wichtigste:
X = obj["drf_wl"]
empty = sum(1 for c in X if len(c) == 0)
print("Empty counters:", empty, "/", len(X))

# Beispiel suchen
for i, c in enumerate(X):
    if len(c) > 0:
        print("First non-empty at idx:", i, "items:", len(c), "total:", sum(c.values()))
        print("Sample:", list(c.items())[:5])
        break
else:
    print("ALL COUNTERS ARE EMPTY in this PKL.")


In [ ]:

pkl = sorted(Path("drf/precomputed_drf_edge").glob("*.pkl"))[0]
obj = pickle.load(open(pkl, "rb"))

print("n_errors:", obj["meta"]["n_errors"])
print("empty:", sum(1 for c in obj["drf_wl"] if len(c)==0), "/", len(obj["drf_wl"]))
print("example total count:", sum(obj["drf_wl"][0].values()))

### ITS–WL Kernel Matrix (edge features)

This heatmap shows the kernel matrix computed using the ITS–WL edge kernel.
Here, reactions are represented by Weisfeiler–Lehman features extracted from the Imaginary Transition State (ITS) graph.

Compared to DRF–WL, the ITS–WL kernel produces a denser similarity structure.
This is expected, as the ITS graph encodes the full combined structure of reactants and products, including unchanged molecular context.

The diagonal again represents self-similarity, while the richer off-diagonal structure indicates that many reactions share common substructures.
As a result, ITS–WL captures broader structural similarity between reactions, not only the explicit reaction center.

In [ ]:
mode = "edge"
n = 200

K_its, y_its_small = build_kernel_matrix_from_loaded(
    X_its, y_its,
    mode=mode,
    n=n,
)

print("ITS kernel matrix stats:", kernel_matrix_stats(K_its))

fig = px.imshow(
    K_its,
    title=f"Kernel Matrix Heatmap (ITS–WL {mode}, n={n})",
    aspect="auto",
)
fig.show()

**Figure (ITS–WL):** Kernel matrix heatmap computed using the ITS–WL edge kernel.
Each entry \(K_{ij}\) corresponds to the multiset intersection of Weisfeiler–Lehman features extracted from the Imaginary Transition State graphs.
Compared to DRF–WL, the ITS–WL kernel exhibits a denser similarity structure, as the ITS graph encodes the full molecular context of reactants and products.
Off-diagonal similarities reflect shared structural motifs beyond the reaction center.

### Comparison of DRF–WL and ITS–WL Kernel Matrices

The DRF–WL and ITS–WL kernel matrices reveal complementary notions of reaction similarity.
DRF–WL focuses exclusively on reaction-specific changes by computing the symmetric difference between reactant and product features.
As a result, the corresponding kernel matrix is sparse, with non-zero similarities only for reactions that share similar bond-change patterns.

In contrast, ITS–WL operates on the Imaginary Transition State graph, which encodes the full structural context of both reactants and products.
This leads to a denser kernel matrix, as reactions may share common substructures even if their reaction centers differ.

Consequently, DRF–WL provides a highly selective notion of similarity tailored to reaction mechanisms,
whereas ITS–WL captures broader structural resemblance between reactions.
Both representations are therefore suitable for different aspects of reaction classification.

**Figure:** Kernel matrix heatmaps for DRF–WL (bottom) and ITS–WL (top) using edge-based Weisfeiler–Lehman features.
Each entry \(K_{ij}\) corresponds to the multiset intersection between the feature representations of reactions \(i\) and \(j\).
The diagonal indicates self-similarity, while off-diagonal values reflect shared structural or reaction-specific features.
DRF–WL produces a sparse kernel emphasizing reaction changes, whereas ITS–WL yields a denser kernel capturing overall structural similarity.

In [ ]:

def upper_triangle_values(K):
    n = K.shape[0]
    return K[np.triu_indices(n, k=1)]

vals_drf = upper_triangle_values(K_drf)  # DRF Kernel-Matrix
vals_its = upper_triangle_values(K_its)  # ITS Kernel-Matrix

fig = px.histogram(
    x=[vals_drf, vals_its],
    labels={"value": "Kernel value", "variable": "Kernel"},
    nbins=50,
    opacity=0.6,
    title="Distribution of Kernel Values: DRF–WL vs ITS–WL",
)

fig.data[0].name = "DRF–WL"
fig.data[1].name = "ITS–WL"
fig.show()

**Figure:** Distribution of off-diagonal kernel values for DRF–WL and ITS–WL.
DRF–WL produces a highly sparse similarity distribution with many zero entries, reflecting its focus on reaction-specific changes.
In contrast, ITS–WL yields a broader distribution, capturing shared structural context between reactions.

## SVM Classification with a Custom Reaction Kernel

An SVM classifier was trained using a custom kernel based on the multiset intersection of reaction features.
Since the kernel operates on pairs of reactions rather than explicit feature vectors, the kernel matrix was precomputed and passed to the SVM using `kernel="precomputed"`.
All classification experiments are conducted using precomputed kernel feature representations.
This enables a fair comparison between DRF–WL and ITS–WL kernels, as the same SVM configuration
and training procedure is applied to both representations.


To systematically evaluate kernel variants, we run the same SVM setup for each feature mode separately.
This yields comparable accuracies for edge-, vertex-, and shortest-path-based WL representations without mixing feature spaces.

### DRF _ EDGE Only

In [ ]:
res = train_svm_with_precomputed_kernel(
    X_drf["edge"],      #DRF–WL Edge-Features
    y_drf["edge"],      #Reaktionsklassen
    n=6000,              #Dataset-Größe 
    test_size=0.2,      #Anteil des Testdatensatzes
    C=1.0,              #Strenge der SVM
    seed=42,            #Fixiert Zufall
)

In [ ]:
res = train_svm_from_precomputed_dir(
    precomp_dir="drf_small/precomputed_drf_edge",  # <-- dein SMALL ordner
    feature_key="drf_wl",
    subset_ids=None,      # nur subset_001..003 (optional)
    n=1000,
    test_size=0.2,
    C=1.0,
    seed=42,
)

In [ ]:
res = train_svm_from_precomputed_dir(
    precomp_dir="drf_big/precomputed_drf_edge",    # <-- dein BIG ordner
    feature_key="drf_wl",
    subset_ids=[1,2,3],        # alle PKLs in dem Ordner
    n=600,
    test_size=0.3,
    C=1.0,
    seed=42,
)

In [ ]:
#
#res_all = run_svm_for_modes(
#    X_drf,              #DRF–WL Edge-Features
#    y_drf,              #Reaktionsklassen
#    n=600,              #Dataset-Größe
#    test_size=0.2,      #Anteil des Testdatensatzes
#    C=1.0,              #Strenge der SVM
#)

In [ ]:
res_its = train_svm_from_precomputed_dir(
    precomp_dir="its_big/precomputed_its_edge",
    feature_key="its_wl",
    subset_ids=None,
    n=2000,
    test_size=0.3,
    C=1.0,
    seed=42,
)

1)

In [ ]:
import time
import pandas as pd

from scripts.wp3_svm import train_svm_from_precomputed_dir
from scripts.wp3_plots import plot_experiment_results

# Pfade (anpassen, falls bei dir anders)
DRF_DIRS = {"edge": "drf_big/precomputed_drf_edge", "vertex": "drf_big/precomputed_drf_vertex", "sp": "drf_big/precomputed_drf_sp"}
ITS_DIRS = {"edge": "its_big/precomputed_its_edge", "vertex": "its_big/precomputed_its_vertex", "sp": "its_big/precomputed_its_sp"}

results = []

def run_one(kernel_name, mode, precomp_dir, feature_key, n, test_size, C=1.0, seed=42):
    t0 = time.time()
    res = train_svm_from_precomputed_dir(
        precomp_dir=precomp_dir,
        feature_key=feature_key,
        subset_ids=None,
        n=n,
        test_size=test_size,
        C=C,
        seed=seed,
        verbose=False,
    )
    dt = time.time() - t0
    return {
        "kernel": kernel_name,
        "mode": mode,
        "n": n,
        "test_size": test_size,
        "C": C,
        "accuracy": float(res.acc),
        "runtime_sec": float(dt),
    }

# Section 4 (Baseline)
results.append(run_one("DRF–WL", "edge", DRF_DIRS["edge"], "drf_wl", n=2000, test_size=0.2))
results.append(run_one("ITS–WL", "edge", ITS_DIRS["edge"], "its_wl", n=2000, test_size=0.2))

df_results = pd.DataFrame(results)
df_results

2)

In [ ]:
for mode in ["vertex", "edge", "sp"]:
    results.append(run_one("DRF–WL", mode, DRF_DIRS[mode], "drf_wl", n=2000, test_size=0.2))
    results.append(run_one("ITS–WL", mode, ITS_DIRS[mode], "its_wl", n=2000, test_size=0.2))

df_results = pd.DataFrame(results)
df_results.tail(6)

Section 6 — Dataset size variation, fix: mode=edge, test=0.2

In [ ]:
for n in [600, 2000, 6000]:
    results.append(run_one("DRF–WL", "edge", DRF_DIRS["edge"], "drf_wl", n=n, test_size=0.2))
    results.append(run_one("ITS–WL", "edge", ITS_DIRS["edge"], "its_wl", n=n, test_size=0.2))

df_results = pd.DataFrame(results)
df_results[df_results["mode"]=="edge"].tail(6)

Section 7 — Train/Test split variation, fix: mode=edge, n=2000

In [ ]:
for ts in [0.2, 0.3, 0.4]:
    results.append(run_one("DRF–WL", "edge", DRF_DIRS["edge"], "drf_wl", n=2000, test_size=ts))
    results.append(run_one("ITS–WL", "edge", ITS_DIRS["edge"], "its_wl", n=2000, test_size=ts))

df_results = pd.DataFrame(results)
df_results[df_results["n"]==2000].tail(6)

Section 8 — Summary table + plots

for ts in [0.2, 0.3, 0.4]:
    results.append(run_one("DRF–WL", "edge", DRF_DIRS["edge"], "drf_wl", n=2000, test_size=ts))
    results.append(run_one("ITS–WL", "edge", ITS_DIRS["edge"], "its_wl", n=2000, test_size=ts))

df_results = pd.DataFrame(results)
df_results[df_results["n"]==2000].tail(6)

In [ ]:
figs = plot_experiment_results(df_results)
for _, fig in figs.items():
    fig.show()

In [ ]:
from pathlib import Path

its_dir = DATA_DIR / "its_small/precomputed_its_edge"

for f in its_dir.glob("*drf*.pkl"):
    print("Deleting:", f.name)
    f.unlink()   # ⚠️ löscht die Datei endgültig

print("Done. All DRF files removed from ITS directory.")

In [ ]:
from pathlib import Path

its_dirs = [
    DATA_DIR / "its_small/precomputed_its_vertex",
    DATA_DIR / "its_small/precomputed_its_edge",
    DATA_DIR / "its_small/precomputed_its_sp",
]

for d in its_dirs:
    for f in d.glob("*drf*.pkl"):
        print("Deleting:", d.name, "->", f.name)
        f.unlink()

print("Done. Removed DRF files from all ITS precomputed directories.")

In [ ]:
subset_ids = [1,2,3,4,5]
n = 600
test_size = 0.2
C = 1.0
seed = 42

for mode, drf_dir, its_dir in [
    ("vertex", DATA_DIR/"drf_small/precomputed_drf_vertex", DATA_DIR/"its_small/precomputed_its_vertex"),
    ("edge",   DATA_DIR/"drf_small/precomputed_drf_edge",   DATA_DIR/"its_small/precomputed_its_edge"),
    ("sp",     DATA_DIR/"drf_small/precomputed_drf_sp",     DATA_DIR/"its_small/precomputed_its_sp"),
]:
    # DRF modes
    res = train_svm_from_precomputed_dir(
        precomp_dir=drf_dir, feature_key="drf_wl", subset_ids=subset_ids,
        n=n, test_size=test_size, C=C, seed=seed
    )
    # speichern…
    add_result("S5_modes", "DRF–WL", mode, n, test_size, C, seed, res)

    # ITS modes
    res = train_svm_from_precomputed_dir(
        precomp_dir=its_dir, feature_key="its_wl", subset_ids=subset_ids,
        n=n, test_size=test_size, C=C, seed=seed
    )
    # speichern…
    add_result("S5_modes", "ITS–WL", mode, n, test_size, C, seed, res)

df_results = pd.DataFrame(results)
df_results[df_results["tag"]=="S5_modes"].sort_values(["kernel","mode"])